In [2]:
import pandas as pd
import numpy as np
import json

In [16]:
myFile = pd.read_csv('D:/project/550_Pattern/Project_1/550-p1-cset-krk-1.csv')
myFile
myFrame=pd.DataFrame(myFile.values, columns = ["White-King-File", "White-King-rank", "White-Rook-file", "White-Rook-rank","Black-King-file","Black-King-rank","Class"])
myFrame

,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
0,c,1,e,2,d,3,draw
1,b,1,d,7,c,7,draw
2,d,2,g,1,g,2,draw
3,d,4,g,6,f,6,draw
4,c,1,b,5,a,5,draw
...,...,...,...,...,...,...,...
214,d,3,d,7,a,7,nine
215,b,1,b,4,g,1,nine
216,d,4,e,8,g,2,nine
217,c,2,d,5,f,1,nine


In [17]:
myFrame.shape

(219, 7)

In [36]:
# Split into training set:60%, holdout set:20%, Validation set: 20%

training_set = myFrame.sample(frac=0.6)
remaining_set = myFrame.drop(training_set.index)
training_set

,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
156,d,3,b,6,a,4,seven
112,d,2,d,8,a,2,five
15,a,1,h,6,g,6,draw
213,d,4,d,8,g,1,nine
87,c,3,g,3,c,1,three
...,...,...,...,...,...,...,...
98,d,2,g,8,b,1,four
80,d,3,h,2,c,1,three
207,c,1,a,5,f,1,nine
198,d,1,c,4,b,7,nine


In [37]:
remaining_set


,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
0,c,1,e,2,d,3,draw
2,d,2,g,1,g,2,draw
6,c,1,g,2,a,1,draw
7,d,2,g,2,f,3,draw
9,b,1,g,5,f,6,draw
...,...,...,...,...,...,...,...
204,c,1,b,5,h,2,nine
205,d,3,e,6,h,3,nine
208,b,2,f,3,h,1,nine
214,d,3,d,7,a,7,nine


In [38]:
classes, cls_counts = np.unique(training_set['Class'], return_counts = True)
classes, cls_counts


(array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
        'three', 'two', 'zero'], dtype=object),
 array([11, 15, 12, 10, 16, 12, 14,  8, 12, 12,  9]))

In [39]:
def calculate_entropy(df_label):
    classes,class_counts = np.unique(df_label,return_counts = True)
    entropy_value = np.sum([(-class_counts[i]/np.sum(class_counts))*np.log2(class_counts[i]/np.sum(class_counts)) 
                        for i in range(len(classes))])
    return entropy_value

In [40]:
calculate_entropy(training_set['Class'])

3.431996424099172

In [41]:

for cols in training_set.columns[:-1]:
    print(cols)

White-King-File
White-King-rank
White-Rook-file
White-Rook-rank
Black-King-file
Black-King-rank


In [42]:
# dataset = training_set
# feature = Feature or attribute of the dataset, i.e column name (training_set.columns[:-1]) E.g White-King-File
# label = Name of Class label colums, i.e. 'Class'

def calculate_information_gain(dataset,feature,label): 
    # Calculate the dataset entropy
    dataset_entropy = calculate_entropy(dataset[label])   
    values,feat_counts= np.unique(dataset[feature],return_counts=True)
    
    # Calculate the weighted feature entropy                                # Call the calculate_entropy function
    weighted_feature_entropy = np.sum([(feat_counts[i]/np.sum(feat_counts))*calculate_entropy(dataset.where(dataset[feature]
                              ==values[i]).dropna()[label]) for i in range(len(values))])    
    feature_info_gain = dataset_entropy - weighted_feature_entropy    
    return feature_info_gain

In [43]:
calculate_information_gain(training_set, 'White-King-File', 'Class')

0.48818783751189043

In [86]:
def create_decision_tree(dataset, df, features, label, parent):
    datum = np.unique(df[label], return_counts=True)
    unique_data = np.unique(dataset[label])
#     print('datum',datum)
#     print('unique_data',unique_data)
    if len(unique_data)<=1:
        return unique_data[0]
    elif len(dataset) == 0:
        return unique_data[np.argmax(datum[1])]
    elif len(features) == 0:
        return parent
    else:
#         print('datum[1]',datum[1])
#         print('np.argmax(datum[1]): ',np.argmax(datum[1]))
        parent = unique_data[np.argmax(datum[1])]
#         
#         print("parent",parent)
        
        item_values =  [calculate_information_gain(dataset, feature, label) for feature in features]
#         print("itemvalues:",item_values)
#         print("features:",features)
        optimal_feature_index = np.argmax(item_values)
        optimal_feature = features[optimal_feature_index]
#         print("optimal_feature",optimal_feature)
        decision_tree = {optimal_feature:{}}
        features = [i for i in features if i!=optimal_feature]
#         print("np.unique(dataset[optimal_feature]): ",np.unique(dataset[optimal_feature]))
#         parent = decision_tree
        for value in np.unique(dataset[optimal_feature]):
            min_data = dataset.where(dataset[optimal_feature] == value).dropna()
            
            min_tree = create_decision_tree(min_data, df, features, label, parent)
            
            decision_tree[optimal_feature][value] = min_tree
        
        return (decision_tree)
        

In [87]:
features = training_set.columns[:-1]
label = 'Class'
parent=None
features

Index(['White-King-File', 'White-King-rank', 'White-Rook-file',
       'White-Rook-rank', 'Black-King-file', 'Black-King-rank'],
      dtype='object')

In [88]:
decision_tree = create_decision_tree(training_set,training_set,features,label,parent)
# for tree in decision_tree:
#     print(tree.keys())
  
# decision_tree.values()

In [89]:
json_object = json.dumps(decision_tree, indent = 2)   
print(json_object)


{
  "Black-King-file": {
    "a": {
      "White-Rook-file": {
        "a": {
          "White-King-File": {
            "c": "zero",
            "d": "six"
          }
        },
        "b": {
          "Black-King-rank": {
            "1": {
              "White-King-File": {
                "c": {
                  "White-King-rank": {
                    "1": "three",
                    "2": "one"
                  }
                },
                "d": "three"
              }
            },
            "2": "one",
            "3": "eight",
            "4": "seven",
            "5": "draw",
            "6": {
              "White-King-File": {
                "b": "seven",
                "c": {
                  "White-King-rank": {
                    "1": "eight",
                    "2": "seven"
                  }
                }
              }
            },
            "8": "seven"
          }
        },
        "c": {
          "White-King-File": {
            "c": 

In [29]:
def predict_decision_tree(test_data, decision_tree):
    for nodes in decision_tree.keys():
        value = test_data[nodes]
        decision_tree = decision_tree[nodes][value]
        prediction = 0
        
        if type(decision_tree) is dict:
            prediction = predict_decision_tree(test_data, decision_tree)
        else:
            prediction = decision_tree
            break
    return prediction

In [35]:
# test_data = pd.Series(remaining_set)
prediction = predict_decision_tree(, decision_tree)

TypeError: 'Series' objects are mutable, thus they cannot be hashed